Nantawat Sukrisunt 6510545543
Naytitorn Chaovirachot 6510545560

In [1]:
!jps

16371 SecondaryNameNode
16003 NameNode
16170 DataNode
16555 ResourceManager
16699 NodeManager
163021 Jps


In [2]:
!source ~/.bashrc

In [3]:
!~/hadoop/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in slave mode
hosts filename                   list of hosts to use in slave mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
    

In [4]:
# List files in /
!~/hadoop/bin/hadoop dfs -ls /


Found 5 items
-rw-r--r--   1 hadoop supergroup      85834 2025-01-20 15:17 /27045.txt
drwxr-xr-x   - hadoop supergroup          0 2025-02-04 20:38 /gutenberg
drwx------   - hadoop supergroup          0 2025-01-20 15:21 /tmp
-rw-r--r--   1 hadoop supergroup 1708674492 2025-02-03 15:32 /yellow_tripdata_2016-01.csv
-rw-r--r--   1 hadoop supergroup  431701740 2025-02-03 14:12 /yellow_tripdata_2016-01.csv.gz


In [5]:
# Delete /gutenberg directory
!~/hadoop/bin/hadoop dfs -rm -r -f  /gutenberg


Deleted /gutenberg


Test your hadoop wordcount

In [6]:
!~/hadoop/bin/hadoop jar /home/hadoop/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar -file ./mapper.py    -mapper ./mapper.py -file ./reducer.py   -reducer ./reducer.py -input /27045.txt -output /gutenberg

2025-02-04 20:40:38,616 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [./mapper.py, ./reducer.py, /tmp/hadoop-unjar1280134622742346177/] [] /tmp/streamjob4516436508246949020.jar tmpDir=null
2025-02-04 20:40:39,456 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
2025-02-04 20:40:39,635 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
2025-02-04 20:40:39,849 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/hadoop/.staging/job_1736778250975_0031
2025-02-04 20:40:39,952 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-02-04 20:40:40,040 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-02-04 20:40:40,061 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostT

In [7]:
#command to bring input file to hdfs
!~/hadoop/bin/hadoop dfs -put yellow_tripdata_2016-01.csv /


put: `/yellow_tripdata_2016-01.csv': File exists


In [8]:
%%writefile mapper_yellow_trip.py
#!/usr/bin/env python3
"""mapper_yellow_trip.py"""

import sys
import gzip
import datetime

# input comes from STDIN (standard input)
for line in sys.stdin:
    fields = line.strip().split(',')  # Split the line by commas (CSV format)

    try:
        pickup_datetime = fields[1]  
        trip_distance = float(fields[4])  
        year = datetime.datetime.strptime(pickup_datetime, "%Y-%m-%d %H:%M:%S").year
        if year == 2016:
            print ('%s\t%s' % (trip_distance, 1))

    except ValueError as e:
        continue  # Skip lines with invalid data

Writing mapper_yellow_trip.py


In [9]:
%%writefile reducer_yellow_trip.py
#!/usr/bin/env python3
"""reducer.py"""

from operator import itemgetter
import sys

total_distance = 0.0
total_trips = 0

# Input comes from STDIN
for line in sys.stdin:
    distance, count = line.strip().split("\t")
    total_distance += float(distance)
    total_trips += int(count)

# Calculate and print the average distance if there are any trips
if total_trips > 0:
    print(f"Average Distance in 2016: {total_distance / total_trips}")
else:
    print("No trips in 2016")

Writing reducer_yellow_trip.py


In [10]:
#command to run your program
!~/hadoop/bin/hadoop jar /home/hadoop/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar -file ./mapper_yellow_trip.py    -mapper ./mapper_yellow_trip.py -file ./reducer_yellow_trip.py   -reducer ./reducer_yellow_trip.py -input /yellow_tripdata_2016-01.csv -output /yellow_trip_output

2025-02-04 20:41:00,389 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [./mapper_yellow_trip.py, ./reducer_yellow_trip.py, /tmp/hadoop-unjar8500934972552678528/] [] /tmp/streamjob2458273501056910097.jar tmpDir=null
2025-02-04 20:41:01,250 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
2025-02-04 20:41:01,426 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
2025-02-04 20:41:01,646 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/hadoop/.staging/job_1736778250975_0032
2025-02-04 20:41:01,751 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-02-04 20:41:01,831 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-02-04 20:41:02,251 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrus

In [14]:
#command to show output
!~/hadoop/bin/hadoop dfs -cat /yellow_trip_output/part-00000


2025-02-04 20:51:09,826 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
Average Distance in 2016: 4.648196988544	
